In [1]:
counts = []
import numpy as np
import pandas as pd
import urllib
for line in urllib.request.urlopen("http://benschmidt.org/sample_page_counts.txt").read().decode('utf-8').split("\n"):
    try:
        counts.append(np.array(list(map(int, line.rstrip().split(',')))))
    except:
        continue
    

In [2]:
print("Testing on {} random books from the SaDDL test corpus".format(len(counts)))

Testing on 56598 random books from the SaaDL test corpus


In [3]:

def current_scheme(pagecounts, chunk_target, overflow_strategy):
        
    cumsums = np.cumsum(pagecounts)
    breaks = np.zeros(cumsums.shape[0], np.int)

    # Store the start of chunks in an array.

    # 1-index the chunk names. 
    breaks[0] = 1

    
    # Last entry gives number of words
    ntokens = cumsums[-1]
    n_chunks = int(int((ntokens) / chunk_target))
    if n_chunks == 0:
        breaks[0] = 1
        return breaks
    # Use actual page counts not including zeros

    overflow = (ntokens % chunk_target)

    if overflow > chunk_target/2:
        overflow -= chunk_target
        n_chunks +=1

    # variable; how far do we want the next one to go?
    if overflow_strategy == "ends":
        target = chunk_target + overflow/2 # + avg_page_n/2
    elif overflow_strategy == "last":
        target = chunk_target
    elif overflow_strategy == "even":
        chunk_target += overflow / n_chunks
        target = chunk_target

    # Proportion of chunk_target that the length adjustment should cap at
    max_adjust = .1 * chunk_target
    # When the remaining words per chunk is higher/lower that x proportion
    #  of the chunk_target, add/remove a chunk.
    new_chunk_threshold = .4 * chunk_target

    i = 1
    while True:
        if i > 10000:
            # I've been getting some infinite loops.
            raise OverflowError("Unable to escape True loop")
        remaining_chunks = n_chunks - i
        if not remaining_chunks:
            break

        last_page = np.argmin(np.abs(cumsums - target))
        if last_page + 1 >= len(breaks):
            break
        if last_page + 2 >= len(breaks) and breaks[-1] == 1:
            # Occurs when the last page is more than 2.5 times the chunk size.
            break
        
        breaks[last_page+1] = 1

        # Remainder adjust - nudge next section slightly, to try to balance
        # out consistently under or oversized parts.
        remaining_nwords = (cumsums[-1] - cumsums[last_page])
        remaining_word_per_chunk_diff = (remaining_nwords / remaining_chunks) - chunk_target
        if abs(remaining_word_per_chunk_diff) > new_chunk_threshold:
            n_chunks += np.sign(remaining_word_per_chunk_diff)

        if overflow_strategy == 'even':
            adjust = remaining_word_per_chunk_diff
        else:
            # Adjust slightly - allowing more adjustment early
            adjust = (0.5+0.5*remaining_chunks/n_chunks) * remaining_word_per_chunk_diff
        if np.abs(adjust) > max_adjust:
            adjust = max_adjust * np.sign(adjust)
        target = chunk_target + cumsums[last_page] + adjust
        i += 1

    return np.cumsum(breaks)

current_scheme(counts[51671], 10000, "even")
counts[51671]

array([  168,    50,   502,   830,    83,   185,   227,   103,   262,
         251,   272,   251,   281,   248,   266,   240,   236,   719,
         536,   295,   312,   271,    63,   312,   654,   386,   250,
         511,   423,   317,   831,   111,   204,   301,   179,   206,
          64,   278,   190,   301,    37,   109,   756,   903,  1117,
         604,  1085,   916,   238,   276,   279,   269,   290,   273,
         308,   283,   294,   279,   271,   234,   944,   589,   256,
         891,   427,   232,   850,   510,   237,   252,    94,   224,
         247,   305,   279,   284,   230,   266,   248,   279,   275,
         304,   323,   318,   296,   297,   291,   177,   201,   250,
         127,   256,   265,   258,   308,   265,   293,   281,   326,
         269,   337,   282,   229,   270,   306,   298,   239,   283,
         276,   286,   282,   324,   249,   251,   294,   160,   217,
         179,   151,   147,   112,   210,  1297,    72,  1441,   263,
        1341,   553,

In [38]:
def end_chunks(page_counts, target, even = False, two_sided = True, procrastinate = False):

    # Register front and back offsets. At the beginning, this is the size of the full page counts;
    # as the process continues, the page_counts object will be slowly trimmed.
    position = [0, len(page_counts)]

    breaks = np.zeros(page_counts.shape[0])
    breaks[0] = 1
    
    loop = -1
       
    while True:
        loop += 1
        if loop > 10000:
            raise OverflowError("Unable to escape loop")
            
        forward = np.cumsum(page_counts)
        if two_sided:
            backward = np.cumsum(np.flip(page_counts))
        
        words_left = forward[-1]
        # Exit conditions
        if words_left < (target * 1.5):
            break
            
        overflow = words_left % target
    
        if (target - overflow) < overflow:
            overflow = -(target - overflow)
            
        if even == True or (even=="mids" and loop > 0):
            chunks_remaining = np.round(words_left/target)
            if chunks_remaining > 2 and two_sided:
                # The share belonging here
                overflow = overflow * 2 / chunks_remaining
            if (chunks_remaining > 1) and (two_sided == False):
                overflow = overflow/chunks_remaining
        # Split the overflow across the ends
        if two_sided:
            loc_target = target + overflow/2
        else:
            loc_target = target + overflow
        if procrastinate:
            # No overflow handling
            loc_target = target
            
        #What is this number supposed to be?    
        if two_sided and words_left < (target * 2.5):
            midpoint = np.argmin(np.abs(forward - words_left/2))
            breaks[midpoint + position[0] + 1]  = 1
            break

        best_front = np.argmin(np.abs(forward - loc_target))
        position[0] = position[0] + best_front + 1
        try:
            breaks[position[0]] = 1
        except IndexError:
            if position[0] == len(breaks):
                # Can happen if the last page is > 2.5x the chunk length
                break
            else:
                raise
        if two_sided:
            best_back = np.argmin(np.abs(backward - loc_target))
            position[1] = position[1] - best_back - 1
            breaks[position[1]] = 1
            new_end = page_counts.shape[0] - best_back - 1
        else:
            # Leave the back for later.
            new_end = page_counts.shape[0] + 1
            
        page_counts = page_counts[(best_front + 1):(new_end)]
        
    return np.cumsum(breaks)


In [5]:
def dumb_chunks(page_counts, target = 10000):
    cumsums = np.cumsum(page_counts)
    n_chunks = np.round(cumsums[-1]/target)
    chunk_size = (cumsums[-1]) // n_chunks + 1
    return cumsums // chunk_size

In [6]:


def test_algorithm(algorithm, print_every = 5000, start_from = 0, **kwargs):
    forward = [[] for i in range(3)]
    backward = [[] for i in range(3)]
    mids = []
    centers = []
    all = []


    work_bias = []

    target = 10000
    vals = []
    for i, p in enumerate(counts[start_from:]):
        if print_every and i % print_every == 0:
            print(i)
        try:
            chunks = algorithm(p, **kwargs)
        except:
            print("error on {}".format(i))
            raise 
        v = pd.DataFrame({'chunk':chunks, "words": p}).groupby('chunk')['words']\
        .agg(['sum', 'count'])['sum']
        f = pd.DataFrame({"error": v.values - target})
        f['chunk'] = range(f.shape[0])
        f['book'] = i
        if len(f) < 3:
            f['which'] = "short"
        else:
            f["which"] = ["first"] + ["mid"] * (f.shape[0] - 2) + ["last"]
        vals.append(f)
    return pd.concat(vals, ignore_index = True)


# Dumb strategy

Divide into chunks and pre-set fixed break points.

In [7]:
all_strategies = []

In [8]:
%%time 

x = test_algorithm(dumb_chunks, target = 10000)
x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])
x['strategy'] = 'Dumb modulo'
all_strategies.append(x)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 25s, sys: 2.68 s, total: 4min 28s
Wall time: 4min 29s


# Current strategy

The current strategy isn't bad.

In [23]:
%%time

x = test_algorithm(current_scheme, start_from = 0, chunk_target = 10000, overflow_strategy="even", print_every = 5000)
x['strategy'] = 'Current even'
all_strategies.append(x)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 33s, sys: 2.92 s, total: 4min 36s
Wall time: 4min 38s


In [10]:
#x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])


### Variation within books

I think this is really important; the current method generally hovers in on an average for any individual book that may not be target chunk size. While overall stats are good, each book is typically 400 words of average chunks off the target, high or low.

In [11]:
x[x.which=='mid'].groupby('book')['error'].agg('mean').reset_index()['error'].agg(['mean', 'max', 'min', 'std'])

mean      -8.627056
max     2285.000000
min    -1754.000000
std      405.470515
Name: error, dtype: float64

# Two-sided end-clobbering

This produces slightly lower std deviations on the main class at the cost of significantly higher in first and last. 

In [12]:
%%time 

x = test_algorithm(end_chunks, target = 10000, even = False, two_sided = True, procrastinate = False, print_every = 5000)
x['strategy'] = 'Two-sided clobber ends'
all_strategies.append(x)
x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 36s, sys: 2.52 s, total: 4min 39s
Wall time: 4min 40s


,mean,std,min,max
which,,,,
first,-23.530499,1448.201613,-3502,10601
last,-22.112185,1452.852939,-3626,17834
mid,-0.002866,275.191191,-5357,6869
short,-967.709331,2998.055992,-9995,4996


### Much less deviation *within* books. 

While the overall stats are similar, this method is much better in that the average deviation by book is much more tightly constrained. Each book's mean error std of 51 words off the target, compared to 414 to the current method.

In [13]:
x[x.which=='mid'].groupby('book')['error'].agg('mean').reset_index()['error'].agg(['mean', 'max', 'min', 'std'])

mean      -0.230268
max      928.000000
min    -5357.000000
std       62.012531
Name: error, dtype: float64

# Start two-sided, then even

Use the first chunk to try and get as close to 10K per chunk in the remaining ones as possible:
then distribute the error using an 'even' strategy.

In [39]:
%%time 

x = test_algorithm(end_chunks, target = 10000, even = "mids", two_sided = True, procrastinate = False, print_every = 5000)
x['strategy'] = 'Two-sided clobber ends then two-sided even'
all_strategies.append(x)
x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 54s, sys: 3.76 s, total: 4min 58s
Wall time: 5min


,mean,std,min,max
which,,,,
first,-23.530499,1448.201613,-3502,10601
last,-22.112185,1452.852939,-3626,17834
mid,-1.679775,228.522201,-5357,6869
short,-967.709331,2998.055992,-9995,4996


### Slightly better even chunking

The stats for even chunking (allocating the remainder slowly) come out a little better than the current method. But the within-book means still show much higher variance.

In [14]:
%%time 

x = test_algorithm(end_chunks, target = 10000, even = True, two_sided = True, procrastinate = False)
x['strategy'] = 'Two-sided even'
all_strategies.append(x)

x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 44s, sys: 2.96 s, total: 4min 47s
Wall time: 4min 49s


,mean,std,min,max
which,,,,
first,-4.834778,435.921087,-2378,10601
last,-4.932052,447.335711,-2063,17834
mid,-4.378145,314.977407,-5529,6869
short,-967.709331,2998.055992,-9995,4996


In [15]:
x[x.which=='mid'].groupby('book')['error'].agg('mean').reset_index()['error'].agg(['mean', 'max', 'min', 'std'])

mean      -5.532545
max     2285.000000
min    -5529.000000
std      410.935109
Name: error, dtype: float64

In [16]:
%%time

# Clobber the middle. A weird strategy
x = test_algorithm(end_chunks, target = 10000, even = False, two_sided = True, procrastinate = True)
x['strategy'] = 'Two-sided single bad middle chunk'
all_strategies.append(x)

x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 31s, sys: 2.56 s, total: 4min 34s
Wall time: 4min 34s


,mean,std,min,max
which,,,,
first,0.159843,161.782300,-4121,10601
last,1.490307,188.954679,-2930,17834
mid,-6.494844,721.889768,-9981,6869
short,-967.709331,2998.055992,-9995,4996


### Here's a clobber the last strategy.

It gets extremely low stds on the first and mid chunks (~222)

In [17]:
%%time

# Clobber the last.
x = test_algorithm(end_chunks, target = 10000, even = False, two_sided = False, procrastinate = True)
x['strategy'] = 'Clobber the last with pointers'
all_strategies.append(x)

x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 29s, sys: 2.42 s, total: 4min 31s
Wall time: 4min 32s


,mean,std,min,max
which,,,,
first,0.040397,161.792021,-4121,10601
last,-61.883499,2883.158454,-6261,17834
mid,-2.539247,218.729868,-9703,6831
short,-973.692478,3164.814230,-9995,4996


In [18]:
x[x.which=='mid'].groupby('book')['error'].agg('mean').reset_index()['error'].agg(['mean', 'max', 'min', 'std'])

mean      -0.469792
max     5221.000000
min    -2553.500000
std       66.782681
Name: error, dtype: float64

### A single-sided even strategy

This works quite well; maybe slightly better than the current single-sided even strategy.

In [19]:
%%time

x = test_algorithm(end_chunks, target = 10000, even = True, two_sided = False, procrastinate = False)
x['strategy'] = 'Single-sided even allocation'
all_strategies.append(x)

x.groupby('which')['error'].agg(['mean', 'std', 'min', 'max'])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
CPU times: user 4min 44s, sys: 2.71 s, total: 4min 47s
Wall time: 4min 48s


,mean,std,min,max
which,,,,
first,-4.834778,435.921087,-2378,10601
last,-3.386105,462.033276,-2197,17834
mid,-4.706673,313.423403,-7961,6831
short,-967.709331,2998.055992,-9995,4996


In [20]:
x[x.which=='mid'].groupby('book')['error'].agg('mean').reset_index()['error'].agg(['mean', 'max', 'min', 'std'])

mean      -5.644320
max     2285.000000
min    -5529.000000
std      410.075084
Name: error, dtype: float64

In [40]:
all_together = pd.concat(all_strategies)

In [41]:
all_together[all_together.which=='mid'].groupby(['book', 'strategy'])['error']\
.agg('mean').reset_index().groupby('strategy')['error']\
.agg(['mean', 'max', 'min', 'std', 'median', 'count'])






,mean,max,min,std,median,count
strategy,,,,,,
Clobber the last with pointers,-0.469792,5221.0,-2553.5,66.782681,-0.266667,46489
Current even,-5.455276,3695.0,-2537.0,409.128668,-6.635379,46477
Dumb modulo,-7.474236,4893.0,-5529.0,411.101962,-7.125000,46477
Single-sided even allocation,-5.644320,2285.0,-5529.0,410.075084,-6.677419,46477
Two-sided clobber ends,-0.230268,928.0,-5357.0,62.012531,0.000000,46477
Two-sided clobber ends then two-sided even,-0.257095,928.0,-5357.0,64.049149,0.000000,46477
Two-sided even,-5.532545,2285.0,-5529.0,410.935109,-6.617647,46477
Two-sided single bad middle chunk,-14.104309,4996.0,-5910.0,886.941209,-8.659574,46477


In [42]:
all_together.groupby(['which', 'strategy'])['error'].agg(['mean', 'std', 'min', 'max'])

mean          std  \
which strategy                                                               
first Clobber the last with pointers                 0.040397   161.792021   
      Current even                                  -4.834778   435.921087   
      Dumb modulo                                 -212.917895   479.651065   
      Single-sided even allocation                  -4.834778   435.921087   
      Two-sided clobber ends                       -23.530499  1448.201613   
      Two-sided clobber ends then two-sided even   -23.530499  1448.201613   
      Two-sided even                                -4.834778   435.921087   
      Two-sided single bad middle chunk              0.159843   161.782300   
last  Clobber the last with pointers               -61.883499  2883.158454   
      Current even                                  -5.234976   471.448498   
      Dumb modulo                                  226.514383   472.391533   
      Single-sided even allocation                  -3.386105   462.033276   
      Two-sided clobber ends                       -22.112185  1452.852939   
      Two-sided clobber ends then two-sided even   -22.112185  1452.852939   
      Two-sided even                                -4.932052   447.335711   
      Two-sided single bad middle chunk              1.490307   188.954679   
mid   Clobber the last with pointers                -2.539247   218.729868   
      Current even                                  -2.906662   321.614276   
      Dumb modulo                                   -4.404729   388.915665   
      Single-sided even allocation                  -4.706673   313.423403   
      Two-sided clobber ends                        -0.002866   275.191191   
      Two-sided clobber ends then two-sided even    -1.679775   228.522201   
      Two-sided even                                -4.378145   314.977407   
      Two-sided single bad middle chunk             -6.494844   721.889768   
short Clobber the last with pointers              -973.692478  3164.814230   
      Current even                               -3130.896513  4646.171505   
      Dumb modulo                                 -149.542366  2107.802006   
      Single-sided even allocation                -967.709331  2998.055992   
      Two-sided clobber ends                      -967.709331  2998.055992   
      Two-sided clobber ends then two-sided even  -967.709331  2998.055992   
      Two-sided even                              -967.709331  2998.055992   
      Two-sided single bad middle chunk           -967.709331  2998.055992   

                                                   min    max  
which strategy                                                 
first Clobber the last with pointers             -4121  10601  
      Current even                               -2378  10601  
      Dumb modulo                                -9914  10609  
      Single-sided even allocation               -2378  10601  
      Two-sided clobber ends                     -3502  10601  
      Two-sided clobber ends then two-sided even -3502  10601  
      Two-sided even                             -2378  10601  
      Two-sided single bad middle chunk          -4121  10601  
last  Clobber the last with pointers             -6261  17834  
      Current even                               -8048  17834  
      Dumb modulo                                -1637  17834  
      Single-sided even allocation               -2197  17834  
      Two-sided clobber ends                     -3626  17834  
      Two-sided clobber ends then two-sided even -3626  17834  
      Two-sided even                             -2063  17834  
      Two-sided single bad middle chunk          -2930  17834  
mid   Clobber the last with pointers             -9703   6831  
      Current even                               -7961   6831  
      Dumb modulo                                -9319   9186  
      Single-sided even allocation               -7961   683

In [57]:
book_lengths = all_together.groupby(['book'])['chunk'].agg({'median':'median'})

with_lengths = all_together.set_index('book').join(book_lengths)
with_lengths[with_lengths['median'] == 4.5].groupby(['which', 'strategy'])['error'].agg(['mean', 'std', 'min', 'max'])

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


mean          std  \
which strategy                                                              
first Clobber the last with pointers                0.428351   114.554554   
      Current even                                  4.448767   297.821246   
      Dumb modulo                                -182.932589   305.471778   
      Single-sided even allocation                  4.448767   297.821246   
      Two-sided clobber ends                       12.733051  1392.453798   
      Two-sided clobber ends then two-sided even   12.733051  1392.453798   
      Two-sided even                                4.448767   297.821246   
      Two-sided single bad middle chunk             0.428351   114.554554   
last  Clobber the last with pointers               29.773112  2778.611963   
      Current even                                  2.849769   312.055845   
      Dumb modulo                                 201.102080   327.643504   
      Single-sided even allocation                  2.849769   312.055845   
      Two-sided clobber ends                        8.722265  1398.246279   
      Two-sided clobber ends then two-sided even    8.722265  1398.246279   
      Two-sided even                               -0.081664   306.959982   
      Two-sided single bad middle chunk            -1.189908   128.760107   
mid   Clobber the last with pointers               -0.629141   115.692009   
      Current even                                  2.233725   304.859507   
      Dumb modulo                                   0.874856   327.773031   
      Single-sided even allocation                  2.233725   304.859507   
      Two-sided clobber ends                        0.464128   141.065187   
      Two-sided clobber ends then two-sided even    0.464128   128.207836   
      Two-sided even                                2.600154   305.267614   
      Two-sided single bad middle chunk             3.241237   704.376524   

                                                   min   max  
which strategy                                                
first Clobber the last with pointers              -705   585  
      Current even                                -766   706  
      Dumb modulo                                -1547   492  
      Single-sided even allocation                -766   706  
      Two-sided clobber ends                     -2640  2773  
      Two-sided clobber ends then two-sided even -2640  2773  
      Two-sided even                              -766   706  
      Two-sided single bad middle chunk           -705   585  
last  Clobber the last with pointers             -5127  4990  
      Current even                               -1125  1010  
      Dumb modulo                                 -476  1982  
      Single-sided even allocation               -1125  1010  
      Two-sided clobber ends                     -2690  2972  
      Two-sided clobber ends then two-sided even -2690  2972  
      Two-sided even                              -797   967  
      Two-sided single bad middle chunk           -682   564  
mid   Clobber the last with pointers             -1205  1126  
      Current even                               -1282  1462  
      Dumb modulo                                -2060  3139  
      Single-sided even allocation               -1282  1462  
      Two-sided clobber ends                     -1205  1126  
      Two-sided clobber ends then two-sided even -1205  1126  
      Two-sided even                             -1205  1248  
      Two-sided single bad middle chunk          -2740  2690